In [1]:
!pip install pyarrow

In [2]:
!pip install fastparquet

In [3]:
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
file_path = "stem_keyword_df4"
df_lemm = pd.read_parquet(file_path)

In [5]:
df_lemm.shape

(45915463, 9)

In [6]:
df_lemm.head()

,keyword,user_id,device_id,product_id,event,event_timestamp,event_date,platform,lemma_keywords
0,remington одежда,,c2fa705c-1453-4114-b02d-44546dd03f88,100068822100,listing_products_view,2024-11-15 10:42:08.995153200+03:00,2024-11-15,iOS,remington одежда
1,remington одежда,,c2fa705c-1453-4114-b02d-44546dd03f88,100068822100,listing_products_view,2024-11-15 10:42:48.993401115+03:00,2024-11-15,iOS,remington одежда
2,remington одежда,,c2fa705c-1453-4114-b02d-44546dd03f88,100068822100,listing_products_view,2024-11-15 10:42:53.970433544+03:00,2024-11-15,iOS,remington одежда
3,armytek,cjsnl3mf2jg5nj2b09g0,3263eefe-016a-405d-bebf-f014528010eb,100068822115,listing_products_view,2024-11-15 19:19:52.380778246+03:00,2024-11-15,iOS,armytek
4,armytek фонарь,cnd51vmf2jgdmh2je7pg,a85a1611-0168-4203-a274-9c1203aef849,100068822115,listing_products_view,2024-11-15 20:18:34.252203800+03:00,2024-11-15,iOS,armytek фонарь


In [7]:
from tqdm.auto import tqdm
tqdm.pandas()

In [8]:
df_lemm['sorted_keywords'] = [' '.join(sorted(query.split())) for query in tqdm(df_lemm['lemma_keywords'], desc="Processing queries")]



Processing queries:   0%|          | 0/45915463 [00:00<?, ?it/s]

In [9]:
df_lemm[['lemma_keywords', 'sorted_keywords']][:20]

,lemma_keywords,sorted_keywords
0,remington одежда,remington одежда
1,remington одежда,remington одежда
2,remington одежда,remington одежда
3,armytek,armytek
4,armytek фонарь,armytek фонарь
5,чехол для ружьё,для ружьё чехол
6,чехол для ружьё,для ружьё чехол
7,чехол для ружьё,для ружьё чехол
8,чехол для ружьё,для ружьё чехол
9,чехол для ружьё,для ружьё чехол


In [10]:
keyword_counts = df_lemm['sorted_keywords'].value_counts().reset_index()
keyword_counts

,sorted_keywords,count
0,машина стиральный,209488
1,телевизор,164980
2,смартфон,158136
3,kuchenland,154305
4,чайник электрический,149850
...,...,...
378112,merrel зимний кроссовок мужской,1
378113,f-t83505,1
378114,2s air dji аккамулятор,1
378115,600013994957,1


In [11]:
keyword_counts.columns = ['sorted_keywords', 'frequency']
keyword_counts

,sorted_keywords,frequency
0,машина стиральный,209488
1,телевизор,164980
2,смартфон,158136
3,kuchenland,154305
4,чайник электрический,149850
...,...,...
378112,merrel зимний кроссовок мужской,1
378113,f-t83505,1
378114,2s air dji аккамулятор,1
378115,600013994957,1


In [12]:
low_freq_threshold = keyword_counts['frequency'].quantile(0.02)
# print(low_freq_threshold)
low_freq_keywords = keyword_counts[keyword_counts['frequency'] <= low_freq_threshold]

print(f"Количество низкочастотных запросов: {len(low_freq_keywords)}")

Количество низкочастотных запросов: 11554


In [13]:
low_freq_keywords[:20]

,sorted_keywords,frequency
366563,10050fe hhy hyundai бензиновый генератор,1
366564,100042536578,1
366565,гринч маска,1
366566,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...",1
366567,металлидти,1
366568,металлидти сольфеджио,1
366569,zofft фен-стайлер,1
366570,гельтечь пилинг сыворотка,1
366571,dell g2524h монитор,1
366572,диван кровать оскар,1


In [21]:
pattern = r"\b\d{12}\b"

# удаляем артикулы
low_freq_keywords = low_freq_keywords[~low_freq_keywords['sorted_keywords'].str.contains(pattern, na=False)]
# 100042536578


In [22]:
low_freq_keywords[:20]

,sorted_keywords,frequency
366563,10050fe hhy hyundai бензиновый генератор,1
366565,гринч маска,1
366566,", , , 32 81 hd sber sdx-32h2128 smart tv смотр...",1
366567,металлидти,1
366568,металлидти сольфеджио,1
366569,zofft фен-стайлер,1
366570,гельтечь пилинг сыворотка,1
366571,dell g2524h монитор,1
366572,диван кровать оскар,1
366573,ra-kv0402s,1


In [24]:
low_freq_keywords.shape

(10631, 2)

In [25]:
low_freq_keywords[:1000].to_csv("low_freq_queries_1000.csv", index=False, encoding="utf-8")